In [7]:
import os
MacOSPath = "/Users/grantseiter/Dropbox/github/Biden-Tax-Proposals/Tax-Calculator-3.0.0/"
MsWinPath = "C:\\Users\\grant.seiter\\Dropbox\\github\\Biden-Tax-Proposals\\Tax-Calculator-3.0.0\\"
os.chdir(MacOSPath)
workdir = os.getcwd()
print (workdir)
from taxcalc import *
import pandas as pd
import numpy as np
import copy 
import matplotlib.pyplot as plt

/Users/grantseiter/Dropbox/github/Biden-Tax-Proposals/Tax-Calculator-3.0.0


### Step 1: Create dataframes of all vars

In [8]:
#  CHNAGE THE BELOW REFORM JSON and YEAR TO DESIRED ANALYSIS FRAMEWORK

cyr = 2030
rec = Records()
pol = Policy()
pol.implement_reform(Policy.read_json_reform('../reforms/biden-iitax-reforms.json'))
calc= Calculator(pol, rec)
calc.advance_to_year(cyr)
calc.calc_all()
df = calc.dataframe([], all_vars=True)

# create variables needed in tau_nc construction outside of loop
df['net_bus_inc'] = (df['e02000'] - df['e26270']) + df['e00900p']

df['mtrC'] = np.where((df['c04800'] > 0) & (df['net_bus_inc'] > 0), calc.mtr(variable_str='e00900p', wrt_full_compensation=False)[1], 0)
df['mtrE1'] = np.where((df['c04800'] > 0) & (df['net_bus_inc'] > 0), calc.mtr(variable_str='e02000', wrt_full_compensation=False)[1], 0)
df['mtrE2'] = np.where((df['c04800'] > 0) & (df['net_bus_inc'] > 0), calc.mtr(variable_str='e26270', wrt_full_compensation=False)[1], 0)
df['mtrE'] = df['mtrE1'] - df['mtrE2']

df['mtr_net_bus_inc'] = df['mtrE'] + df['mtrC']
df['weight_net_bus_inc'] = np.where((df['c04800'] > 0) & (df['net_bus_inc']>0), df['net_bus_inc']/(df['net_bus_inc'].sum()),0)
df['w_net_bus_inc'] = np.where((df['c04800'] > 0) & (df['net_bus_inc']>0), df['mtr_net_bus_inc'] * df['weight_net_bus_inc'],0)

# loop for rest of variables
def make_mtrs(df, var):
    df['mtr_' + var] = np.where((df['c04800'] > 0) & (df[var]>0), calc.mtr(variable_str=var, wrt_full_compensation=False)[1], 0)
    df['weight_' + var] = np.where((df['c04800'] > 0) & (df[var]>0), df[var]/(df[var].sum()),0)
    df['w_' + var] = np.where((df['c04800'] > 0) & (df[var]>0), df['mtr_' + var] * df['weight_' + var],0)
    return df

for var in ['e00650', 'e00300', 'p22250', 'p23250', 'e01700','e00200p']:
    make_mtrs(df, var)

### Step 2: Create tau parameters from dataframe

In [9]:
# create weights
df = df.assign(wage_weight = (df['e00200p'] * df['s006']) / sum(df['e00200p'] * df['s006']))
df = df.assign(interest_weight = (df['e00300'] * df['s006']) / sum(df['e00300'] * df['s006']))
df = df.assign(dividend_weight = (df['e00650'] * df['s006']) / sum(df['e00650'] * df['s006']))
df = df.assign(ltgains_weight = (df['p23250'] * df['s006']) / sum(df['p23250'] * df['s006']))
df = df.assign(stgains_weight = (df['p22250'] * df['s006']) / sum(df['p22250'] * df['s006']))
df = df.assign(pension_weight = (df['e01700'] * df['s006']) / sum(df['e01700'] * df['s006']))
df = df.assign(business_weight = (df['net_bus_inc'] * df['s006']) / sum(df['net_bus_inc'] * df['s006']))

# 
df.loc[(df['e00200p'] > 0) & (df['c04800'] > 0), 'wage_weight'] = (df.loc[(df['e00200p'] > 0) & (df['c04800'] > 0), 'e00200p'] * df.loc[(df['e00200p'] > 0) & (df['c04800'] > 0), 's006']) \
/ sum(df.loc[(df['e00200p'] > 0) & (df['c04800'] > 0), 'e00200p'] * df.loc[(df['e00200p'] > 0) & (df['c04800'] > 0), 's006'])

df.loc[(df['e00300'] > 0) & (df['c04800'] > 0), 'interest_weight'] = (df.loc[(df['e00300'] > 0) & (df['c04800'] > 0), 'e00300'] * df.loc[(df['e00300'] > 0) & (df['c04800'] > 0), 's006']) \
/ sum(df.loc[(df['e00300'] > 0) & (df['c04800'] > 0), 'e00300'] * df.loc[(df['e00300'] > 0) & (df['c04800'] > 0), 's006'])

df.loc[(df['e00650'] > 0) & (df['c04800'] > 0), 'dividend_weight'] = (df.loc[(df['e00650'] > 0) & (df['c04800'] > 0), 'e00650'] * df.loc[(df['e00650'] > 0) & (df['c04800'] > 0), 's006']) \
/ sum(df.loc[(df['e00650'] > 0) & (df['c04800'] > 0), 'e00650'] * df.loc[(df['e00650'] > 0) & (df['c04800'] > 0), 's006'])

df.loc[(df['p23250'] > 0) & (df['c04800'] > 0), 'ltgains_weight'] = (df.loc[(df['p23250'] > 0) & (df['c04800'] > 0), 'p23250'] * df.loc[(df['p23250'] > 0) & (df['c04800'] > 0), 's006']) \
/ sum(df.loc[(df['p23250'] > 0) & (df['c04800'] > 0), 'p23250'] * df.loc[(df['p23250'] > 0) & (df['c04800'] > 0), 's006'])

df.loc[(df['p22250'] > 0) & (df['c04800'] > 0), 'stgains_weight'] = (df.loc[(df['p22250'] > 0) & (df['c04800'] > 0), 'p22250'] * df.loc[(df['p22250'] > 0) & (df['c04800'] > 0), 's006']) \
/ sum(df.loc[(df['p22250'] > 0) & (df['c04800'] > 0), 'p22250'] * df.loc[(df['p22250'] > 0) & (df['c04800'] > 0), 's006'])

df.loc[(df['e01700'] > 0) & (df['c04800'] > 0), 'pension_weight'] = (df.loc[(df['e01700'] > 0) & (df['c04800'] > 0), 'e01700'] * df.loc[(df['e01700'] > 0) & (df['c04800'] > 0), 's006']) \
/ sum(df.loc[(df['e01700'] > 0) & (df['c04800'] > 0), 'e01700'] * df.loc[(df['e01700'] > 0) & (df['c04800'] > 0), 's006'])

df.loc[(df['net_bus_inc'] > 0) & (df['c04800'] > 0), 'business_weight'] = (df.loc[(df['net_bus_inc'] > 0) & (df['c04800'] > 0), 'net_bus_inc'] * df.loc[(df['net_bus_inc'] > 0) & (df['c04800'] > 0), 's006']) \
/ sum(df.loc[(df['net_bus_inc'] > 0) & (df['c04800'] > 0), 'net_bus_inc'] * df.loc[(df['net_bus_inc'] > 0) & (df['c04800'] > 0), 's006'])

# sum of (weight x mtr)
results = {
    'tau_wages': sum(df.loc[(df['e00200p'] > 0) & (df['c04800'] > 0), 'mtr_e00200p'] * df.loc[(df['e00200p'] > 0) & (df['c04800'] > 0), 'wage_weight']),
    'tau_interest': sum(df.loc[(df['e00300'] > 0) & (df['c04800'] > 0), 'mtr_e00300'] * df.loc[(df['e00300'] > 0) & (df['c04800'] > 0), 'interest_weight']),
    'tau_dividends': sum(df.loc[(df['e00650'] > 0) & (df['c04800'] > 0), 'mtr_e00650'] * df.loc[(df['e00650'] > 0) & (df['c04800'] > 0), 'dividend_weight']),
    'tau_ltcapgain' : sum(df.loc[(df['p23250'] > 0) & (df['c04800'] > 0), 'mtr_p23250'] * df.loc[(df['p23250'] > 0) & (df['c04800'] > 0), 'ltgains_weight']),
    'tau_stcapgain': sum(df.loc[(df['p22250'] > 0) & (df['c04800'] > 0), 'mtr_p22250'] * df.loc[(df['p22250'] > 0) & (df['c04800'] > 0), 'stgains_weight']),
    'tau_taxdef': sum(df.loc[(df['e01700'] > 0) & (df['c04800'] > 0), 'mtr_e01700'] * df.loc[(df['e01700'] > 0) & (df['c04800'] > 0), 'pension_weight']),
    'tau_businc': sum(df.loc[(df['net_bus_inc'] > 0) & (df['c04800'] > 0), 'mtr_net_bus_inc'] * df.loc[(df['net_bus_inc'] > 0) & (df['c04800'] > 0), 'business_weight'])
}

results

{'tau_wages': 0.2615570975722658,
 'tau_interest': 0.3501197033722555,
 'tau_dividends': 0.23576880808217743,
 'tau_ltcapgain': 0.302696552046099,
 'tau_stcapgain': 0.360018242929579,
 'tau_taxdef': 0.2519422134315737,
 'tau_businc': 0.28690800605154393}

In [ ]:
# AS OF 10/01

# 2021 Output -- Biden 
{ 'tau_wages': 0.22579741535205383,
 'tau_businc': 0.23688645926320534
 'tau_dividends': 0.22154782524778652,
    'tau_interest': 0.32587731116122715,
     'tau_stcapgain': 0.35316762250245715,
 'tau_ltcapgain': 0.30317417181222733,
 'tau_taxdef': 0.20557529276761868,

# 2030 Output -- Biden 
{'tau_wages': 0.2615570975722658,
    'tau_businc': 0.28690800605154393}
 'tau_dividends': 0.23576880808217743,
 'tau_interest': 0.3501197033722555,
 'tau_stcapgain': 0.360018242929579,
 'tau_ltcapgain': 0.302696552046099,
 'tau_taxdef': 0.2519422134315737,
 
 # 2021 Output - Current Law
{'tau_wages': 0.22055633310698056,
 'tau_businc': 0.21212348629625527}
 'tau_dividends': 0.18144637902683441,
 'tau_interest': 0.3045090753423554,
 'tau_stcapgain': 0.33172716752431924,
 'tau_ltcapgain': 0.21411727367834893,
 'tau_taxdef': 0.20432708930036364,

# 2030 Output - Current Law
{'tau_wages': 0.26270810021063296,
 'tau_businc': 0.2826226943704139}
 'tau_dividends': 0.19913431879864069,
 'tau_interest': 0.34051886072557286,
 'tau_stcapgain': 0.3519825834480394,
 'tau_ltcapgain': 0.21957756590835306,
 'tau_taxdef': 0.2517037725957203,